In [1]:
import moldesign as mdt
from moldesign import units as u
import ipywidgets as ipy

OpenMM could not be imported; using remote docker container
PDBFixer could not be imported; using remote docker container
PySCF not installed; using remote docker container
Reading configuration from /Users/t_leeday/.moldesign/moldesign.yml


In [2]:
from moldesign.models.lammps_model import LAMMPSPotential
import parmed as med
# mdt.configure()

In [3]:
mdt.Atom

moldesign.molecules.atoms.Atom

In [4]:
mol = mdt.from_pdb('1yu8')

In [5]:
mol = mdt.Molecule([res for res in mol.residues if res.type == "protein"])

In [6]:
mdmol = mdt.assign_forcefield(mol)

print "Done parameterizing"

Connecting to docker host at unix://var/run/docker.sock... done


Widget Javascript not detected.  It may not be installed or enabled properly.


Done parameterizing


In [7]:
rs = mdt.widgets.ResidueSelector(mdmol)
rs

Widget Javascript not detected.  It may not be installed or enabled properly.


In [9]:
canvas = mdmol.draw()

In [10]:
mdmol.set_energy_model(mdt.models.LAMMPSInteractive, affected_atom=rs.selected_atoms[0])
mdmol.calculate()
mdmol.set_integrator(integrator=mdt.integrators.LAMMPSNvt, timestep=1.0*u.fs, 
                     temperature=300.0*u.kelvin, frame_interval=20.0*u.fs)

LAMMPS output is captured by PyLammps wrapper


MolecularProperties({'positions': <Quantity([[ 10.367      29.903      18.753    ]
 [  9.6081971  30.4339133  19.1560511]
 [ 10.3127893  29.9718557  17.7468089]
 ..., 
 [  8.242      10.473      15.05     ]
 [  7.004      10.292      15.052    ]
 [  8.877      11.103      14.169    ]], 'ang')>, 'potential_energy': <Quantity(-4354.13539812, 'kcalpermol')>, 'forces': <Quantity([[-22.01450917 -39.97082631  15.02823307]
 [ -3.75360413  -4.4598381    3.53402641]
 [ 10.36030096   8.57141355   2.72603121]
 ..., 
 [ -4.10467713  11.80953405  -0.20540562]
 [  9.94705074  -5.52989577  16.52583564]
 [  2.38928098 -12.58205002  17.15230766]], 'kcalpermol / ang')>})

In [14]:
def apply_user_force(x=0.0, y=-10.0, z=0.0):
    vec = [x, y, z]
    if all(v==0.0 for v in vec):
        return;
    vec = vec * u.angstrom
    mdmol.energy_model.apply_force(vec)
    mytraj = mdmol.run(100*u.fs)
    for step in mytraj:
        mdmol.energy_model.render_molecule(traj=step)
        canvas.set_positions(step.positions.value_in(u.angstrom))
#     for step in xrange(10):
#         mdmol.run(20 * u.fs)
#         mdmol.energy_model.render_molecule()
#         canvas.set_positions()
    mdmol.energy_model.finish_interaction()
    canvas.set_positions()
    
    mdmol.set_energy_model(mdt.models.LAMMPSInteractive, affected_atom=rs.selected_atoms[0])
    mdmol.calculate()

In [15]:
canvas.set_positions()
canvas.stick()
canvas

In [16]:
ipy.interact(apply_user_force, x=ipy.FloatSlider(min=-10.0, max=10.0, step=1.0, continuous_update=False),
            y=ipy.FloatSlider(min=-10.0, max=10.0, step=1.0, continuous_update=False),
            z=ipy.FloatSlider(min=-10.0, max=10.0, step=1.0, continuous_update=False))

[0.0010676248888888892, 0.017081998222222228, 0.0024021560000000003]


In [13]:


# mdmol.set_integrator(integrator=mdt.integrators.LAMMPSNvt, timestep=1.0*u.fs, 
#                      temperature=300.0*u.kelvin, frame_interval=10.0*u.fs)
# mdmol.energy_model.apply_force([10.0, 0.0, 0.0])
# my_traj = mdmol.run(200 * u.fs)

# canvas.stick()
# canvas

In [14]:
# for traj in my_traj:
#     mdmol.energy_model.render_molecule(traj)
#     canvas.set_positions(positions=traj.positions.value_in(u.angstrom))
    


In [ ]:
# mdmol.energy_model.finish_interaction()
# canvas.set_positions()